In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time, os
import numpy as np
import requests
import re
import random

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [4]:
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

In [5]:
def get_property_urls(link):
    # omg thank god for some dude on github for this req_header 0 CAPTCHAS i owe you my life
    
    req_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.8',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
    }


    r = requests.get(link, headers=req_headers)
    soup = BeautifulSoup(r.content, 'html.parser')

    table = soup.find('ul', class_='photo-cards photo-cards_wow photo-cards_short')
    li_tags = table.find_all('li')
    properties = [item.find('div', class_='list-card-info')for item in li_tags]

    property_urls = []

    for item in properties:
        if item != None:
            property_extensions = item.find('a')['href']
            property_urls.append(property_extensions)
            
    return property_urls

In [6]:
# def get_property_features(link):
#     try:
    
#         req_headers = {
#             'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
#             'accept-encoding': 'gzip, deflate, br',
#             'accept-language': 'en-US,en;q=0.8',
#             'upgrade-insecure-requests': '1',
#             'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
#         }


#         r = requests.get(link, headers=req_headers)
#         soup = BeautifulSoup(r.content, 'html.parser')
#         facts_and_features_table = soup.find('div', class_='ds-home-facts-and-features reso-facts-features sheety-facts-features')


#         # this is the code for extracting features our of the home fact list table
#         all_features = facts_and_features_table.find_all('li', class_='ds-home-fact-list-item')
#         features_list = [item.text.split(':') for item in all_features]
#         price = soup.find('div', class_='ds-home-details-chip').find('span', class_='ds-value').text
#         features_list.append(['Total Price', price])

#         # this is code for extracting number of bedroom/baths from a separate div
#         # then formatting it to match our features from the first table and appending
#         # to feature list
#         beds_baths = soup.find(text='Bedrooms and bathrooms').next_element.text
#         beds_baths = re.split('(\d+)',beds_baths.replace('1/2', 'Half'))[:-1]
#         bed_baths_formatted = [i+j for i,j in zip(beds_baths[::2], beds_baths[1::2])]


#         for item in bed_baths_formatted:
#             features_list.append(item.split(':'))

#         print(features_list)
#         return features_list
#     except:
#         return None


In [7]:
def get_property_features(link):
    try:
    
        req_headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'en-US,en;q=0.8',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
        }


        r = requests.get(link, headers=req_headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        facts_and_features_table = soup.find('div', class_='ds-home-facts-and-features reso-facts-features sheety-facts-features')


        # this is the code for extracting features our of the home fact list table
        all_features = facts_and_features_table.find_all('li', class_='ds-home-fact-list-item')
        features_list = [item.text.split(':') for item in all_features]

        address = soup.find('h1', class_='ds-address-container').text.replace(',\xa0', ', ')
        
        # so im adding this if sold parameter meaning are we looking at sold property urls
        # trying this out but the idea is that we zillow has properties under the sold filter
        # that have been listed again and we dont want to collect those potentially duplicate
        # data points.
        price = soup.find('div', class_='ds-home-details-chip').find('span', class_='ds-value').text
            
        summary_info = soup.find_all('span', class_='ds-bed-bath-living-area')

        # getting the third element in summnary info which is square feet and then splitting/
        # reversing it so its formatted as ['Square Feet', '728'] and not ['728', 'square feet']
        for item in summary_info:
            if 'sqft' in item.text.lower() or 'square feet' in item.text.lower():
                sqft = item.text.split(" ", 1)[::-1]
            else:
                sqft = np.nan

        # getting all text out of neighborhoods stats about market trends bit weird
        # but we only certain things so sorry about the weird splitting
#         neighborhood_stats = soup.find('span',{'id':'skip-link-neighborhood'}).next_element.get_text()
#         neighborhood_stats = neighborhood_stats.split('Neighborhood stats')[1].split('Nearby homes')[0]
#         neighborhood_stats_formatted = re.sub(r'[^\x00-\x7f]',r'', neighborhood_stats) 
        
        # trying neighborhood stats in new way if this doesnt work uncomment code above
        neighborhood_stats_formatted = soup.find(text = 'Neighborhood stats').next_element.get_text()

        # finding if building was recently constructed or renovated from construction details
        # table 
        construction_details = soup.find_all('span', class_='Text-aiai24-0 cZksOw')
        for item in construction_details:
            if 'new construction' in item.text.lower():
                construction = item.text.split(':')


        # get estimated monthly cost table this only works with chrome driver kill me...
#         time.sleep(.5+3*random.random())
#         driver = webdriver.Chrome(chromedriver)
#         driver.get(link)
#         chrome_soup = BeautifulSoup(driver.page_source, 'html.parser')
#         monthly_cost_table = chrome_soup.find('div',class_='sc-qYIQh fmfdy').get_text().split('Chevron Down')[0:-1]
#         for item in monthly_cost_table:
#             if '$' in item:
#                 features_list.append(item.split('$'))
#             else:
#                 features_list.append(item.split('ities'))
                
#         driver.close()

        features_list.append(['Total Price', price])
        features_list.append(['Address', address])
        features_list.append(sqft)
        features_list.append(construction)

        # this is code for extracting number of bedroom/baths from a separate div
        # then formatting it to match our features from the first table and appending
        # to feature list
        beds_baths = soup.find(text='Bedrooms and bathrooms').next_element.text
        beds_baths = re.split('(\d+)',beds_baths.replace('1/2', 'Half'))[:-1]
        bed_baths_formatted = [i+j for i,j in zip(beds_baths[::2], beds_baths[1::2])]

        for item in bed_baths_formatted:
            features_list.append(item.split(':'))

        features_list.append(['neighborhood_stats', neighborhood_stats_formatted])
        features_list.append(['URL', link])
        
        print(features_list)
        return dict(features_list)
    except:
        return None

In [18]:

def get_property_features2(link):
    try:
    
        req_headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'en-US,en;q=0.8',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
        }


        r = requests.get(link, headers=req_headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        facts_and_features_table = soup.find('div', class_='ds-home-facts-and-features reso-facts-features sheety-facts-features')


        # this is the code for extracting features our of the home fact list table
        all_features = facts_and_features_table.find_all('li', class_='ds-home-fact-list-item')
        features_list = [item.text.split(':') for item in all_features]

        address = soup.find('h1', class_='ds-address-container').text.replace(',\xa0', ', ')
        
        # so im adding this if sold parameter meaning are we looking at sold property urls
        # trying this out but the idea is that we zillow has properties under the sold filter
        # that have been listed again and we dont want to collect those potentially duplicate
        # data points.
        price = soup.find('span', class_='sc-pscky hEQFyS ds-status-details').get_text().lower().split(':')
        
        if 'sold' in price:

            summary_info = soup.find_all('span', class_='ds-bed-bath-living-area')

            # getting the third element in summnary info which is square feet and then splitting/
            # reversing it so its formatted as ['Square Feet', '728'] and not ['728', 'square feet']
            for item in summary_info:
                if 'sqft' in item.text.lower() or 'square feet' in item.text.lower():
                    sqft = item.text.split(" ", 1)[::-1]
                else:
                    sqft = np.nan

            # getting all text out of neighborhoods stats about market trends 
            neighborhood_stats_formatted = soup.find(text = 'Neighborhood stats').next_element.get_text()

            # finding if building was recently constructed or renovated from construction details
            construction_details = soup.find_all('span', class_='Text-aiai24-0 cZksOw')
            for item in construction_details:
                if 'new construction' in item.text.lower():
                    construction = item.text.split(':')

            features_list.append(['Total Price', price[1]])
            features_list.append(['Address', address])
            features_list.append(sqft)
            features_list.append(construction)

            # this is code for extracting number of bedroom/baths from a separate div
            # then formatting it to match our features from the first table and appending
            # to feature list
            beds_baths = soup.find(text='Bedrooms and bathrooms').next_element.text
            beds_baths = re.split('(\d+)',beds_baths.replace('1/2', 'Half'))[:-1]
            bed_baths_formatted = [i+j for i,j in zip(beds_baths[::2], beds_baths[1::2])]

            for item in bed_baths_formatted:
                features_list.append(item.split(':'))

            features_list.append(['neighborhood_stats', neighborhood_stats_formatted])
            features_list.append(['URL', link])

            print(features_list)
            return dict(features_list)
        else:
            pass
    except:
        return None

In [19]:
def generate_page_urls(location, status):
    """
    Description: get urls for first 20 pages for a location
    Parameter 1: location (str): e.g. 'chicago-il'
    Parameter 2: status (str): 'sold' or 'for_sale'
    
    Return: first 20 page urls (list)
    """
    
    if status == 'sold':
        base_url = 'https://www.zillow.com/{}/sold/'.format(location)
    elif status == 'for_sale':
        base_url = 'https://www.zillow.com/homes/{}/'.format(location)
    
#     https://www.zillow.com/chicago-il/sold/5_p/
    first_20_pages_ext = [base_url + str(i) + '_p' for i in range(2,21)]
    # as ganesh pointed out im not adding url for first page so im appending it here...
    first_20_pages_ext.append(base_url)
    return first_20_pages_ext
    

In [20]:
def get_all_property_urls(first_20_pages_ext):
    
    """
    Description: Get all the property urls for each page on Zillow
    Parameters: first 20 pages of urls (list)
    Return: urls for every property on each of the 20 pages
    """
    
    all_pages_property_urls = []
    for link in first_20_pages_ext:
        all_pages_property_urls.append(get_property_urls(link))

    # so right now i have a list of list and i want to flatten it for convenience
    all_urls = np.array(all_pages_property_urls).flatten()
    return all_urls

In [11]:
first_20_pages_ext = generate_page_urls('chicago-il', 'sold')
all_urls = get_all_property_urls(first_20_pages_ext)

In [ ]:
########################## testing ##############################

In [ ]:
# req_headers = {
#     'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
#     'accept-encoding': 'gzip, deflate, br',
#     'accept-language': 'en-US,en;q=0.8',
#     'upgrade-insecure-requests': '1',
#     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
# }

# link = first_20_pages_ext[0]

# r = requests.get(link, headers=req_headers)
# soup = BeautifulSoup(r.content, 'html.parser')

# test = soup.find_all('div', class_='list-card-info')
# for item in test:
#     print(item.a['href'])
# # table = soup.find('ul', class_='photo-cards photo-cards_wow photo-cards_short')
# # li_tags = table.find_all('li')
# # properties = [item.find('div', class_='list-card-info')for item in li_tags]

# # property_urls = []

# # for item in properties:
# #     if item != None:
# #         property_extensions = item.find('a')['href']
# #         property_urls.append(property_extensions)

In [16]:
test_url = 'https://www.zillow.com/homedetails/10617-S-Avenue-O-Chicago-IL-60617/2081127043_zpid/'
r = requests.get(test_url, headers=req_headers)
soup = BeautifulSoup(r.content, 'html.parser')
facts_and_features_table = soup.find('div', class_='ds-home-facts-and-features reso-facts-features sheety-facts-features')


# this is the code for extracting features our of the home fact list table
all_features = facts_and_features_table.find_all('li', class_='ds-home-fact-list-item')
features_list = [item.text.split(':') for item in all_features]

address = soup.find('h1', class_='ds-address-container').text.replace(',\xa0', ', ')
# soup.find('span', class_='ds-status-icon zsg-icon-recently-sold').get_text()
price = soup.find('span', class_='sc-pscky hEQFyS ds-status-details').get_text().split(':')
# price = soup.find('div', class_='ds-home-details-chip').find('span', class_='ds-value').text
summary_info = soup.find_all('span', class_='ds-bed-bath-living-area')

# getting the third element in summnary info which is square feet and then splitting/
# reversing it so its formatted as ['Square Feet', '728'] and not ['728', 'square feet']

for item in summary_info:
    if 'sqft' in item.text.lower() or 'square feet' in item.text.lower():
        sqft = item.text.split(" ", 1)[::-1]

# print(sqft)
        
#         sqft = summary_info[2].text.split(" ", 1)[::-1]
# else:
#     sqft = np.nan
# getting all text out of neighborhoods stats about market trends bit weird
# but we only certain things so sorry about the weird splitting
# neighborhood_stats = soup.find('span',{'id':'skip-link-neighborhood'}).next_element.get_text()
# neighborhood_stats = neighborhood_stats.split('Neighborhood stats')[1].split('Nearby homes')[0]
# neighborhood_stats_formatted = re.sub(r'[^\x00-\x7f]',r'', neighborhood_stats) 

neighborhood_stats = soup.find(text = 'Neighborhood stats').next_element.get_text()

# finding if building was recently constructed or renovated from construction details
# table 
construction_details = soup.find_all('span', class_='Text-aiai24-0 cZksOw')
for item in construction_details:
    if 'new construction' in item.text.lower():
        construction = item.text.split(':')
        
        
# get estimated monthly cost table trying this with chrome driver fingers crossed...
# driver = webdriver.Chrome(chromedriver)
# driver.get(test_url)
# chrome_soup = BeautifulSoup(driver.page_source, 'html.parser')
# monthly_cost_table = chrome_soup.find('div',class_='sc-qYIQh fmfdy').get_text().split('Chevron Down')[0:-1]
# for item in monthly_cost_table:
#     if '$' in item:
#         features_list.append(item.split('$'))
#     else:
#         features_list.append(item.split('ities'))

# driver.close()
        
features_list.append(['Total Price', price[1]])
features_list.append(['Address', address])
features_list.append(sqft)
features_list.append(construction)

# this is code for extracting number of bedroom/baths from a separate div
# then formatting it to match our features from the first table and appending
# to feature list
beds_baths = soup.find(text='Bedrooms and bathrooms').next_element.text
beds_baths = re.split('(\d+)',beds_baths.replace('1/2', 'Half'))[:-1]
bed_baths_formatted = [i+j for i,j in zip(beds_baths[::2], beds_baths[1::2])]

for item in bed_baths_formatted:
    features_list.append(item.split(':'))
    
features_list.append(['neighborhood_stats', neighborhood_stats])



In [17]:
features_list

[['Type', 'Multi Family'],
 ['Year built', '1905'],
 ['Heating', 'No Data'],
 ['Cooling', 'No Data'],
 ['Parking', '2 spaces'],
 ['Total Price', ' $145,000'],
 ['Address', '10617 S Avenue O, Chicago, IL 60617'],
 ['sqft', '--'],
 ['New construction', ' No'],
 ['Bedrooms', ' 5'],
 ['Bathrooms', ' 3'],
 ['Full bathrooms', ' 2'],
 ['Half bathrooms', ' 1'],
 ['neighborhood_stats',
  'Home values in East Side have risen 0.6\u200a% (↑) over the past 12 months.Zillow predicts the home values in East Side will rise 6.4% (↑) in the next year.This home is valued 12.5% higher (↑) than the median home in East Side.The median Zestimate® for this neighborhood is $128,946.']]

In [ ]:
########################## testing end ##############################

In [21]:
data_list = []
bum_links = []
for link in all_urls:
    print(link)
    data_raw = get_property_features2(link)
    if data_raw != None:
        # adding the dict down here so we dont throw an error when something
        # doesnt come up/can have conditions
        data_list.append(data_raw)
    else:
        # creating this so we can see what links didnt work and look into why later
        bum_links.append(link)
        

https://www.zillow.com/homedetails/1728-W-Foster-Ave-APT-2-Chicago-IL-60640/61959521_zpid/
https://www.zillow.com/homedetails/2771-W-Francis-Pl-UNIT-204N-Chicago-IL-60647/101328437_zpid/
https://www.zillow.com/homedetails/1301-W-Madison-St-APT-506-Chicago-IL-60607/62013992_zpid/
https://www.zillow.com/homedetails/714-S-Dearborn-St-FL-5-Chicago-IL-60605/2084488168_zpid/
https://www.zillow.com/homedetails/4002-N-Southport-Ave-2-Chicago-IL-60613/2112471163_zpid/
https://www.zillow.com/homedetails/30-E-Huron-St-APT-1110-Chicago-IL-60611/70468919_zpid/
https://www.zillow.com/homedetails/728-W-Jackson-Blvd-C58-Chicago-IL-60661/49911039_zpid/
https://www.zillow.com/homedetails/10617-S-Avenue-O-Chicago-IL-60617/2081127043_zpid/
https://www.zillow.com/homedetails/1620-S-Michigan-Ave-UNIT-407-Chicago-IL-60616/87714385_zpid/
https://www.zillow.com/homedetails/5402-S-East-View-Park-3-Chicago-IL-60615/3988525_zpid/
[['Type', 'Condo'], ['Year built', '1912'], ['Heating', 'Other'], ['Cooling', 'Wall'

https://www.zillow.com/homedetails/182-W-Lake-St-APT-2203-Chicago-IL-60601/87719872_zpid/
https://www.zillow.com/homedetails/7232-W-Ibsen-St-Chicago-IL-60631/3511314_zpid/
https://www.zillow.com/homedetails/5135-S-Kenwood-Ave-APT-203-Chicago-IL-60615/3986294_zpid/
https://www.zillow.com/homedetails/211-E-Ohio-St-APT-609-Chicago-IL-60611/58368114_zpid/
https://www.zillow.com/homedetails/1733-W-Irving-Park-Rd-APT-213-Chicago-IL-60613/3709294_zpid/
https://www.zillow.com/homedetails/6345-S-Lavergne-Ave-Chicago-IL-60638/3949143_zpid/
https://www.zillow.com/homedetails/1435-E-72nd-St-Chicago-IL-60619/49909756_zpid/
https://www.zillow.com/homedetails/10848-S-Green-St-Chicago-IL-60643/4145639_zpid/
https://www.zillow.com/homedetails/407-W-116th-St-Chicago-IL-60628/4153817_zpid/
https://www.zillow.com/homedetails/7517-S-Wabash-Ave-Chicago-IL-60619/4005357_zpid/
[['Type', 'Single Family'], ['Year built', '1924'], ['Heating', 'Gas'], ['Cooling', 'No Data'], ['Parking', '2 spaces'], ['Lot', '4,02

[['Type', 'Multi Family'], ['Year built', '1900'], ['Heating', 'Forced air'], ['Cooling', 'No Data'], ['Parking', 'None'], ['Total Price', ' $293,000'], ['Address', '6908 S Union Ave, Chicago, IL 60621'], ['sqft', '--'], ['New construction', ' No'], ['Bedrooms', ' 5'], ['Bathrooms', ' 3'], ['Full bathrooms', ' 3'], ['neighborhood_stats', 'Home values in Englewood have risen 0.6\u200a% (↑) over the past 12 months.Zillow predicts the home values in Englewood will increase 9% (↑) in the next year.This home is valued 501.1% higher (↑) than the median home in Englewood.The median Zestimate® for this neighborhood is $48,744.'], ['URL', 'https://www.zillow.com/homedetails/6908-S-Union-Ave-Chicago-IL-60621/2079760214_zpid/']]
https://www.zillow.com/homedetails/10350-S-Indiana-Ave-Chicago-IL-60628/4138442_zpid/
https://www.zillow.com/homedetails/5549-S-Kilbourn-Ave-Chicago-IL-60629/3937454_zpid/
[['Type', 'Single Family'], ['Year built', '1950'], ['Heating', 'Forced air'], ['Cooling', 'Central'

[['Type', 'Multi Family'], ['Year built', '1906'], ['Heating', 'Forced air'], ['Cooling', 'No Data'], ['Parking', '2 spaces'], ['Total Price', ' $375,000'], ['Address', '2442 N Avers Ave, Chicago, IL 60647'], ['sqft', '--'], ['New construction', ' No'], ['Bedrooms', ' 4'], ['Bathrooms', ' 3'], ['Full bathrooms', ' 3'], ['neighborhood_stats', 'Home values in Logan Square have risen 0.6\u200a% (↑) over the past 12 months.Zillow predicts the home values in Logan Square will increase 1.6% (↑) in the next year.This home is valued 7.1% lower (↓) than the median home in Logan Square.The median Zestimate® for this neighborhood is $403,670.'], ['URL', 'https://www.zillow.com/homedetails/2442-N-Avers-Ave-Chicago-IL-60647/2143703973_zpid/']]
https://www.zillow.com/homedetails/2680-N-Orchard-St-UNIT-2-Chicago-IL-60614/2078307962_zpid/
https://www.zillow.com/homedetails/5135-W-Carmen-Ave-Chicago-IL-60630/3638065_zpid/
[['Type', 'Single Family'], ['Year built', '2020'], ['Heating', 'Forced air'], ['

https://www.zillow.com/homedetails/10918-S-Edbrooke-Ave-Chicago-IL-60628/4140236_zpid/
https://www.zillow.com/homedetails/936-W-Montana-St-302-Chicago-IL-60614/2078867958_zpid/
https://www.zillow.com/homedetails/5158-S-Ashland-Ave-Chicago-IL-60609/2078671692_zpid/
https://www.zillow.com/homedetails/6171-N-Sheridan-Rd-APT-1305-Chicago-IL-60660/3691620_zpid/
[['Type', 'Condo'], ['Year built', '1970'], ['Heating', 'Baseboard'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['HOA', '$285/month'], ['Total Price', ' $104,500'], ['Address', '6171 N Sheridan Rd APT 1305, Chicago, IL 60660'], ['Square Feet', '447'], ['New construction', ' No'], ['Bedrooms', ' 0'], ['Bathrooms', ' 1'], ['Full bathrooms', ' 1'], ['neighborhood_stats', 'Home values in Edgewater have risen 0.6\u200a% (↑) over the past 12 months.Zillow predicts the home values in Edgewater will increase 1.3% (↑) in the next year.This home is valued 56.9% lower (↓) than the median home in Edgewater.The median Zestimate® for this n

...

https://www.zillow.com/homedetails/1111-N-Western-Ave-APT-3S-Chicago-IL-60622/89907713_zpid/
https://www.zillow.com/homedetails/4017-N-Mason-Ave-Chicago-IL-60634/3653318_zpid/
[['Type', 'Single Family'], ['Year built', '1922'], ['Heating', 'Forced air'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['HOA', 'None'], ['Lot', '3,750 sqft'], ['Total Price', ' $265,000'], ['Address', '4017 N Mason Ave, Chicago, IL 60634'], ['Square Feet', '1,458'], ['New construction', ' No'], ['Bedrooms', ' 3'], ['Bathrooms', ' 1'], ['Full bathrooms', ' 1'], ['neighborhood_stats', 'Home values in Avondale have risen 0.6\u200a% (↑) over the past 12 months.Zillow predicts the home values in Avondale will increase 0% (↑) in the next year.This home is valued 32.9% lower (↓) than the median home in Avondale.The median Zestimate® for this neighborhood is $394,918.'], ['URL', 'https://www.zillow.com/homedetails/4017-N-Mason-Ave-Chicago-IL-60634/3653318_zpid/']]
https://www.zillow.com/homedetails/2900-N-Richmo

[['Type', 'Condo'], ['Year built', '2006'], ['Heating', 'Forced air'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['HOA', '$150/month'], ['Total Price', ' $345,000'], ['Address', '1813 W Grand Ave #3, Chicago, IL 60622'], ['Square Feet', '1,200'], ['New construction', ' No'], ['Bedrooms', ' 2'], ['Bathrooms', ' 2'], ['Full bathrooms', ' 2'], ['neighborhood_stats', 'Home values in West Town have risen 0.6\u200a% (↑) over the past 12 months.Zillow predicts the home values in West Town will rise 1.7% (↑) in the next year.This home is valued 32% lower (↓) than the median home in West Town.The median Zestimate® for this neighborhood is $507,475.'], ['URL', 'https://www.zillow.com/homedetails/1813-W-Grand-Ave-3-Chicago-IL-60622/65550998_zpid/']]
https://www.zillow.com/homedetails/303-W-Ohio-St-APT-3507-Chicago-IL-60654/113955337_zpid/
[['Type', 'Condo'], ['Year built', '2009'], ['Heating', 'Forced air'], ['Cooling', 'Central'], ['Parking', '1 space'], ['HOA', '$797/month'], ['Total Pri

https://www.zillow.com/homedetails/805-W-Oakdale-Ave-APT-1A-Chicago-IL-60657/3728506_zpid/
https://www.zillow.com/homedetails/5233-N-La-Crosse-Ave-Chicago-IL-60630/3637262_zpid/
[['Type', 'Single Family'], ['Year built', '1948'], ['Heating', 'Forced air'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['HOA', 'None'], ['Lot', '3,781 sqft'], ['Total Price', ' $450,000'], ['Address', '5233 N La Crosse Ave, Chicago, IL 60630'], ['Square Feet', '1,700'], ['New construction', ' No'], ['Bedrooms', ' 3'], ['Bathrooms', ' 3'], ['Full bathrooms', ' 2'], ['Half bathrooms', ' 1'], ['neighborhood_stats', 'Home values in Forest Glen have risen 0.6\u200a% (↑) over the past 12 months.Zillow predicts the home values in Forest Glen will fall 2.3% (↓) in the next year.This home is valued 21.7% higher (↑) than the median home in Forest Glen.The median Zestimate® for this neighborhood is $369,613.'], ['URL', 'https://www.zillow.com/homedetails/5233-N-La-Crosse-Ave-Chicago-IL-60630/3637262_zpid/']]
http

[['Type', 'Single Family'], ['Year built', '1929'], ['Heating', 'Forced air'], ['Cooling', 'Central'], ['Parking', '2 spaces'], ['HOA', 'None'], ['Lot', '3,780 sqft'], ['Total Price', ' $362,000'], ['Address', '4551 N Mobile Ave, Chicago, IL 60630'], ['Square Feet', '2,000'], ['New construction', ' No'], ['Bedrooms', ' 4'], ['Bathrooms', ' 2'], ['Full bathrooms', ' 2'], ['neighborhood_stats', 'Home values in Jefferson Park have risen 0.6\u200a% (↑) over the past 12 months.Zillow predicts the home values in Jefferson Park will decrease 2.4% (↓) in the next year.This home is valued 19.4% higher (↑) than the median home in Jefferson Park.The median Zestimate® for this neighborhood is $303,165.'], ['URL', 'https://www.zillow.com/homedetails/4551-N-Mobile-Ave-Chicago-IL-60630/3650726_zpid/']]
https://www.zillow.com/homedetails/5321-N-Manila-Ave-Chicago-IL-60630/2078404764_zpid/
[['Type', 'Multi Family'], ['Year built', '1924'], ['Heating', 'Forced air'], ['Cooling', 'No Data'], ['Parking', 

https://www.zillow.com/homedetails/617-N-Latrobe-Ave-Chicago-IL-60644/2132184446_zpid/
https://www.zillow.com/homedetails/2850-N-Christiana-Ave-Chicago-IL-60618/87713304_zpid/
https://www.zillow.com/homedetails/1148-W-Roscoe-St-APT-1W-Chicago-IL-60657/80849390_zpid/
https://www.zillow.com/homedetails/1151-E-61st-St-3-Chicago-IL-60637/63706271_zpid/
https://www.zillow.com/homedetails/1872-N-Clybourn-Ave-APT-401-Chicago-IL-60614/3736312_zpid/
https://www.zillow.com/homedetails/4542-N-Kenton-Ave-Chicago-IL-60630/2092083046_zpid/
https://www.zillow.com/homedetails/4110-N-Pontiac-Ave-Chicago-IL-60634/3590682_zpid/
https://www.zillow.com/homedetails/4106-N-Mozart-St-APT-GE-Chicago-IL-60618/2078740306_zpid/
https://www.zillow.com/homedetails/2927-W-36th-St-Chicago-IL-60632/3842229_zpid/
https://www.zillow.com/homedetails/3963-W-Belmont-Ave-UNIT-344-Chicago-IL-60618/113963263_zpid/
https://www.zillow.com/homedetails/5050-S-East-End-Ave-APT-3D-Chicago-IL-60615/2089435498_zpid/
https://www.zillo

In [22]:
pd.DataFrame(data_list)

,Type,Year built,Heating,Cooling,Parking,HOA,Total Price,Address,Square Feet,New construction,Bedrooms,Bathrooms,Full bathrooms,neighborhood_stats,URL,Lot,sqft,Half bathrooms
0,Condo,1912,Other,Wall,No Data,$485/month,"$174,900","5402 S East View Park #3, Chicago, IL 60615",900,No,1,1,1,Home values in East Hyde Park have risen 0.6 %...,https://www.zillow.com/homedetails/5402-S-East...,NaN,NaN,NaN
1,Single Family,1895,Forced air,No Data,None,NaN,"$22,000","11624 S Yale Ave, Chicago, IL 60628",880,No,4,1,1,Home values in West Pullman have risen 0.6 % (...,https://www.zillow.com/homedetails/11624-S-Yal...,"4,650 sqft",NaN,NaN
2,Multi Family,1918,Forced air,Central,2 spaces,NaN,"$480,000","3330 W Hutchinson St, Chicago, IL 60618","1,000",No,7,4,4,Home values in Irving Park have risen 0.6 % (↑...,https://www.zillow.com/homedetails/3330-W-Hutc...,NaN,NaN,NaN
3,Condo,1965,Baseboard,No Data,1 space,$260/month,"$139,000","7418 N Harlem Ave #3, Chicago, IL 60631",NaN,No,2,1,1,Home values in Edison Park have risen 0.6 % (↑...,https://www.zillow.com/homedetails/7418-N-Harl...,NaN,--,NaN
4,Single Family,1956,Forced air,Central,2 spaces,None,"$329,000","3934 N Oriole Ave, Chicago, IL 60634","1,250",No,4,2,2,Home values in Belmont Heights have risen 0.6 ...,https://www.zillow.com/homedetails/3934-N-Orio...,"3,750 sqft",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Condo,1926,No Data,No Data,None,$282/month,"$173,000","1221 W Lunt Ave APT 2B, Chicago, IL 60626",900,No,2,1,1,Home values in Rogers Park have risen 0.6 % (↑...,https://www.zillow.com/homedetails/1221-W-Lunt...,NaN,NaN,NaN
304,Multi Family,1924,Other,No Data,3 spaces,NaN,"$625,000","4941 N Francisco Ave, Chicago, IL 60625","1,500",No,5,3,3,Home values in Ravenswood have risen 0.6 % (↑)...,https://www.zillow.com/homedetails/4941-N-Fran...,NaN,NaN,NaN
305,Condo,1987,Forced air,Central,1 space,$344/month,"$173,000","3660 N Lake Shore Dr APT 2809, Chicago, IL 60613",550,No,0,1,1,Home values in Lake View East have risen 0.6 %...,https://www.zillow.com/homedetails/3660-N-Lake...,NaN,NaN,NaN
306,Condo,1974,Gas,Central,1 space,$394/month,"$212,500","1540 N La Salle Dr APT 303, Chicago, IL 60610",NaN,No,1,1,1,Home values in Old Town have risen 0.6 % (↑) o...,https://www.zillow.com/homedetails/1540-N-La-S...,NaN,--,NaN


In [23]:
# data_list
df = pd.DataFrame(data_list)
# df.to_csv('austin_data.csv', header=True)

In [24]:
df.columns.to_list()

['Type',
 'Year built',
 'Heating',
 'Cooling',
 'Parking',
 'HOA',
 'Total Price',
 'Address',
 'Square Feet',
 'New construction',
 'Bedrooms',
 'Bathrooms',
 'Full bathrooms',
 'neighborhood_stats',
 'URL',
 'Lot',
 'sqft',
 'Half bathrooms']

In [25]:
df.drop(columns=['sqft'], inplace=True)

In [29]:
# df.to_csv('chicago_sold_unclean.csv')

In [120]:
for_sale = pd.read_csv('chicago_for_sale_unclean.csv')
sold = pd.read_csv('chicago_sold_unclean.csv')

In [121]:
sold.columns.to_list()

['Unnamed: 0',
 'Type',
 'Year built',
 'Heating',
 'Cooling',
 'Parking',
 'HOA',
 'Total Price',
 'Address',
 'Square Feet',
 'New construction',
 'Bedrooms',
 'Bathrooms',
 'Full bathrooms',
 'neighborhood_stats',
 'URL',
 'Lot',
 'Half bathrooms',
 'Status']

In [122]:
for_sale.drop(columns=['Price/sqft'], inplace=True)
# sold.shape[0] + for_sale.shape[0]

In [123]:
chicago_full = pd.concat([for_sale, sold]).drop(columns=['Unnamed: 0']).reset_index()
chicago_full

,index,Type,Year built,Heating,Cooling,Parking,Total Price,Address,New construction,Bedrooms,Bathrooms,Full bathrooms,neighborhood_stats,URL,Lot,Square Feet,HOA,Half bathrooms,Status
0,0,Multi Family,1921,Forced air,No Data,2 spaces,"$279,900","5931 S Campbell Ave, Chicago, IL 60629",No,6,3.0,3.0,Home values in Chicago Lawn have risen 0.6% ()...,https://www.zillow.com/homedetails/5931-S-Camp...,NaN,NaN,NaN,NaN,For Sale
1,1,Single Family,1924,Gas,Central,2 spaces,"$138,970","11117 S Sawyer Ave, Chicago, IL 60655",No,3,1.0,1.0,Home values in Mount Greenwood have risen 0.6%...,https://www.zillow.com/homedetails/11117-S-Saw...,"3,789 sqft",988,NaN,NaN,For Sale
2,2,Multi Family,1930,Forced air,No Data,None,"$99,900","7151 S Green St, Chicago, IL 60621",No,6,2.0,2.0,Home values in Englewood have risen 0.6% () ov...,https://www.zillow.com/homedetails/7151-S-Gree...,NaN,NaN,NaN,NaN,For Sale
3,3,Single Family,1898,Gas,No Data,1 space,"$39,900","620 W 60th St, Chicago, IL 60621",No,4,2.0,2.0,Home values in Englewood have risen 0.6% () ov...,https://www.zillow.com/homedetails/620-W-60th-...,"3,301 sqft","1,498",None,NaN,For Sale
4,4,Condo,1934,Forced air,No Data,None,"$168,900","5148 N Monitor Ave APT 202, Chicago, IL 60630",No,2,1.0,1.0,Home values in Jefferson Park have risen 0.6% ...,https://www.zillow.com/homedetails/5148-N-Moni...,NaN,"1,000",$234/month,NaN,For Sale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,303,Condo,1926,No Data,No Data,None,"$173,000","1221 W Lunt Ave APT 2B, Chicago, IL 60626",No,2,1.0,1.0,Home values in Rogers Park have risen 0.6 % (↑...,https://www.zillow.com/homedetails/1221-W-Lunt...,NaN,900,$282/month,NaN,Sold
1085,304,Multi Family,1924,Other,No Data,3 spaces,"$625,000","4941 N Francisco Ave, Chicago, IL 60625",No,5,3.0,3.0,Home values in Ravenswood have risen 0.6 % (↑)...,https://www.zillow.com/homedetails/4941-N-Fran...,NaN,"1,500",NaN,NaN,Sold
1086,305,Condo,1987,Forced air,Central,1 space,"$173,000","3660 N Lake Shore Dr APT 2809, Chicago, IL 60613",No,0,1.0,1.0,Home values in Lake View East have risen 0.6 %...,https://www.zillow.com/homedetails/3660-N-Lake...,NaN,550,$344/month,NaN,Sold
1087,306,Condo,1974,Gas,Central,1 space,"$212,500","1540 N La Salle Dr APT 303, Chicago, IL 60610",No,1,1.0,1.0,Home values in Old Town have risen 0.6 % (↑) o...,https://www.zillow.com/homedetails/1540-N-La-S...,NaN,NaN,$394/month,NaN,Sold


In [124]:
chicago_full.drop_duplicates(subset=['URL'], inplace=True)

In [125]:
chicago_full.drop(columns=['index'], inplace=True)
# chicago_full.to_csv('chicago_full_unclean.csv')

In [ ]:
# df.drop(columns=['Date available', 'Pets', 'Laundry',  'Deposit & fees'], inplace=True)
# commenting this out for austin data but need it for chicago data

In [57]:
# cleaning data below

In [126]:
df = chicago_full

In [127]:
df.replace(to_replace=['None', 'No Data'], value=np.nan, inplace=True)

In [128]:
# df['Price/sqft'] = df['Price/sqft'].apply(lambda x: float(re.sub("[^0-9]", "", x)) if isinstance(x, str) else x)

In [129]:
df['Total Price'] = df['Total Price'].apply(lambda x: float(re.sub("[^0-9]", "", x)) if isinstance(x, str) else x)

In [130]:
df['HOA'] = df['HOA'].apply(lambda x: float(re.sub("[^0-9]", "", x)) if isinstance(x, str) else x)

In [131]:
df['Bedrooms'] = df['Bedrooms'].apply(lambda x: float(re.sub("[^0-9]", "", x)) if isinstance(x, str) else x)

In [132]:
df['Bathrooms'] = df['Bathrooms'].apply(lambda x: float(re.sub("[^0-9]", "", x)) if isinstance(x, str) else x)

In [133]:
df['Square Feet'] = df['Square Feet'].apply(lambda x: float(re.sub("[^0-9]", "", x)) if isinstance(x, str) else x)


In [134]:

# will have to think about how to encode this...
df.Parking.unique()

array(['2 spaces', nan, '1 space', '3 spaces', '4 spaces', '6 spaces',
       '42 spaces', 'Attached Garage', '5 spaces', '7 spaces', '8 spaces',
       'On street', 'Detached Garage', '15 spaces'], dtype=object)

In [135]:
df['Full bathrooms'] = df['Full bathrooms'].apply(lambda x: int(x) if isinstance(x, str) else x)

In [136]:
def acres_to_sqft(x):
    if isinstance(x, list):
        if 'acres' in x:
            sqft = float(x[0]) * 43560
            return round(sqft, 2)
        else:
            return float(x[0].replace(',',''))
    else:
        return x

In [137]:
# making all string units lowercase because of Acres and acres discrepancy 
df['Lot'] = df['Lot'].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [138]:
df['Lot Temp'] = df['Lot'].str.split(' ')
df['Lot'] = df['Lot Temp'].apply(acres_to_sqft)
df.drop(columns=['Lot Temp'], inplace=True)

In [139]:
df.Lot.to_frame()

,Lot
0,NaN
1,3789.0
2,NaN
3,3301.0
4,NaN
...,...
1084,NaN
1085,NaN
1086,NaN
1087,NaN


In [140]:
# lot info is sometimes in acres so we'll have to account for that before running this

# df['Lot'] = df['Lot'].apply(lambda x: float(re.sub("[^0-9]", "", x)) if isinstance(x, str) else x)

In [141]:
df['Half bathrooms'].fillna(0, inplace=True)
df['Half bathrooms'] = df['Half bathrooms'].astype(int)

In [142]:
df['Year built'] = df['Year built'].apply(lambda x: int(x) if isinstance(x, str) else x)

In [143]:
df['Cooling'] = df.Cooling.replace('Refrigerator', 'Refrigeration')

In [144]:
df.Cooling.fillna('None', inplace=True)

In [146]:
df.to_csv('chicago_full_clean.csv')

In [155]:
df

,Type,Year built,Heating,Cooling,Parking,Total Price,Address,New construction,Bedrooms,Bathrooms,Full bathrooms,neighborhood_stats,URL,Lot,Square Feet,HOA,Half bathrooms,Status
0,Multi Family,1921.0,Forced air,None,2 spaces,279900.0,"5931 S Campbell Ave, Chicago, IL 60629",No,6,3.0,3.0,Home values in Chicago Lawn have risen 0.6% ()...,https://www.zillow.com/homedetails/5931-S-Camp...,NaN,NaN,NaN,0,For Sale
1,Single Family,1924.0,Gas,Central,2 spaces,138970.0,"11117 S Sawyer Ave, Chicago, IL 60655",No,3,1.0,1.0,Home values in Mount Greenwood have risen 0.6%...,https://www.zillow.com/homedetails/11117-S-Saw...,3789.0,988.0,NaN,0,For Sale
2,Multi Family,1930.0,Forced air,None,NaN,99900.0,"7151 S Green St, Chicago, IL 60621",No,6,2.0,2.0,Home values in Englewood have risen 0.6% () ov...,https://www.zillow.com/homedetails/7151-S-Gree...,NaN,NaN,NaN,0,For Sale
3,Single Family,1898.0,Gas,None,1 space,39900.0,"620 W 60th St, Chicago, IL 60621",No,4,2.0,2.0,Home values in Englewood have risen 0.6% () ov...,https://www.zillow.com/homedetails/620-W-60th-...,3301.0,1498.0,NaN,0,For Sale
4,Condo,1934.0,Forced air,None,NaN,168900.0,"5148 N Monitor Ave APT 202, Chicago, IL 60630",No,2,1.0,1.0,Home values in Jefferson Park have risen 0.6% ...,https://www.zillow.com/homedetails/5148-N-Moni...,NaN,1000.0,234.0,0,For Sale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,Condo,1926.0,NaN,None,NaN,173000.0,"1221 W Lunt Ave APT 2B, Chicago, IL 60626",No,2,1.0,1.0,Home values in Rogers Park have risen 0.6 % (↑...,https://www.zillow.com/homedetails/1221-W-Lunt...,NaN,900.0,282.0,0,Sold
1085,Multi Family,1924.0,Other,None,3 spaces,625000.0,"4941 N Francisco Ave, Chicago, IL 60625",No,5,3.0,3.0,Home values in Ravenswood have risen 0.6 % (↑)...,https://www.zillow.com/homedetails/4941-N-Fran...,NaN,1500.0,NaN,0,Sold
1086,Condo,1987.0,Forced air,Central,1 space,173000.0,"3660 N Lake Shore Dr APT 2809, Chicago, IL 60613",No,0,1.0,1.0,Home values in Lake View East have risen 0.6 %...,https://www.zillow.com/homedetails/3660-N-Lake...,NaN,550.0,344.0,0,Sold
1087,Condo,1974.0,Gas,Central,1 space,212500.0,"1540 N La Salle Dr APT 303, Chicago, IL 60610",No,1,1.0,1.0,Home values in Old Town have risen 0.6 % (↑) o...,https://www.zillow.com/homedetails/1540-N-La-S...,NaN,NaN,394.0,0,Sold


### ok so now we have cleaned data for austin and chicago dfs now we can encode

In [147]:
sold_only = df

In [148]:
object_df = sold_only.select_dtypes(include=['object'])
object_df.head(3)

,Type,Heating,Cooling,Parking,Address,New construction,neighborhood_stats,URL,Status
0,Multi Family,Forced air,None,2 spaces,"5931 S Campbell Ave, Chicago, IL 60629",No,Home values in Chicago Lawn have risen 0.6% ()...,https://www.zillow.com/homedetails/5931-S-Camp...,For Sale
1,Single Family,Gas,Central,2 spaces,"11117 S Sawyer Ave, Chicago, IL 60655",No,Home values in Mount Greenwood have risen 0.6%...,https://www.zillow.com/homedetails/11117-S-Saw...,For Sale
2,Multi Family,Forced air,None,NaN,"7151 S Green St, Chicago, IL 60621",No,Home values in Englewood have risen 0.6% () ov...,https://www.zillow.com/homedetails/7151-S-Gree...,For Sale


In [149]:
sold_only['Square Feet']

0          NaN
1        988.0
2          NaN
3       1498.0
4       1000.0
         ...  
1084     900.0
1085    1500.0
1086     550.0
1087       NaN
1088    3500.0
Name: Square Feet, Length: 1069, dtype: float64

In [156]:
test = pd.get_dummies(object_df[['Type', 'Cooling', 'New construction', 'Status']], drop_first=True)
test

,Type_Condo,Type_Multi Family,Type_Multiple Occupancy,Type_Single Family,Type_Townhouse,"Cooling_Central, Other",Cooling_None,Cooling_Other,Cooling_Refrigeration,Cooling_Wall,New construction_ Yes,Status_Sold
0,0,1,0,0,0,0,1,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,1,0,0,0,0,0
3,0,0,0,1,0,0,1,0,0,0,0,0
4,1,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1084,1,0,0,0,0,0,1,0,0,0,0,1
1085,0,1,0,0,0,0,1,0,0,0,0,1
1086,1,0,0,0,0,0,0,0,0,0,0,1
1087,1,0,0,0,0,0,0,0,0,0,0,1


In [157]:
continous_df = sold_only.select_dtypes(include=['float64', 'int64'])

In [158]:
# import seaborn as sns
# sns.pairplot(continous_df)

In [159]:
clean_df = continous_df.join(test)
correlations = clean_df.corr()['Total Price'].to_frame()

In [160]:
correlations.sort_values(by=['Total Price'],ascending=False)

,Total Price
Total Price,1.000000
Square Feet,0.669794
Bathrooms,0.644490
Full bathrooms,0.559804
Half bathrooms,0.393242
HOA,0.297219
Bedrooms,0.255721
New construction_ Yes,0.218102
Year built,0.142014
Lot,0.067041
